In [1]:
!pip install capstone
!pip install pefile

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import sys
from capstone import *
from capstone.x86 import *
import pefile
import pandas as pd
import os.path
from os import listdir
import pprint
import json

In [3]:
def get_main_code_section(sections, base_of_code):
    addresses = []
    for section in sections: 
        addresses.append(section.VirtualAddress)
    if base_of_code in addresses:
        #if sections[addresses.index(base_of_code)].Characteristics == int(0x60000020):
        if 1==1:    
            return sections[addresses.index(base_of_code)]
        else:
            return None
    else:
        addresses.append(base_of_code)
        addresses.sort()
        if addresses.index(base_of_code)!= 0:
            #if sections[addresses.index(base_of_code)-1].Characteristics == int(0x60000020):
            if 1==1: 
                return sections[addresses.index(base_of_code)-1]
            else:
                return None
        else:
            return None

In [4]:
def fine_disassemble(exe):
    main_code = get_main_code_section(exe.sections, exe.OPTIONAL_HEADER.BaseOfCode)
    md = Cs(CS_ARCH_X86, CS_MODE_32)
    md.detail = True
    last_address = 0
    last_size = 0
    begin = main_code.PointerToRawData
    end = begin+main_code.SizeOfRawData
    ins_count = 0
    size_count = 0
    instructions = {}
    sequence = []
    pred_name = None
    while True:
        data = exe.get_memory_mapped_image()[begin:end]
        for i in md.disasm(data, begin):
            if i.mnemonic in instructions.keys():
                instructions[i.mnemonic]['count'] += 1
                instructions[i.mnemonic]['frequence'] += 1
                if ins_count - instructions[i.mnemonic]['count'] > instructions[i.mnemonic]['largest_period']:
                    instructions[i.mnemonic]['largest_period'] = instructions[i.mnemonic]['count'] 
                elif ins_count - instructions[i.mnemonic]['count'] < instructions[i.mnemonic]['smallest_period']:
                    instructions[i.mnemonic]['smallest_period'] = instructions[i.mnemonic]['count']
                instructions[i.mnemonic]['count'] = ins_count
            else:
                instructions[i.mnemonic]={
                           'registers':{}, 
                           'count':0,
                           'largest_period':0, 
                           'smallest_period':1000000,
                           'frequence':1,
                           'preds':{},
                           'sucs':{},}        
            if pred_name is not None and pred_name in instructions[i.mnemonic]['preds'].keys():
                instructions[i.mnemonic]['preds'][pred_name] += 1
            elif pred_name is not None:
                instructions[i.mnemonic]['preds'][pred_name] = 1
            if pred_name is not None and i.mnemonic in instructions[pred_name]['sucs'].keys():
                instructions[pred_name]['sucs'][i.mnemonic] += 1
            elif pred_name is not None:
                instructions[pred_name]['sucs'][i.mnemonic] = 1
            for op in i.operands:
                if op.type == X86_OP_REG:
                    if i.reg_name(op.reg) in instructions[i.mnemonic]['registers'].keys():
                        instructions[i.mnemonic]['registers'][i.reg_name(op.reg)] += 1
                    else:
                        instructions[i.mnemonic]['registers'][i.reg_name(op.reg)] = 1
            pred_name = i.mnemonic
            sequence.append(i.mnemonic)
            last_address = int(i.address)
            last_size = i.size
            ins_count += 1
            size_count += last_size
        begin = max(int(last_address),begin)+last_size+1
        if begin >= end:
            print("out")
            break
    return instructions, sequence

In [ ]:
#Paths For Files
malware_base_dir = '../input/firstpart/samples/'
exceptions_counter = 0
counter = 1
exceptions = []
hashes_file = open('../input/hashes61/hashes6.txt')
hashes = hashes_file.readlines()
malware_list = pd.read_csv('../input/malwarelist/malware_list_sample_1.csv', index_col='Hash Of Malware')
stats = {}
a = 2
for f in hashes:
    if malware_list.loc[f[:-1]]['Status']=='clear':
        counter += 1
        try:
            exe = pefile.PE(os.path.join(malware_base_dir,f[:-1]))
            try:
                inst, seq = fine_disassemble(exe)
                stats[f[:-1]]={'stats':inst, 'sequence':seq}
                inst = None
                seq = None
            except:
                exceptions.append([f[:-1],"code section not found"])
        except Exception:
            exceptions_counter += 1
            exceptions.append([f[:-1],"bad format"])
        print(counter)
        if(counter > 300 * a):
            with open('opcodes6%d.json' % counter, 'a') as js:
                json.dump(stats, js)
            stats = {}
            a += 1
print(exceptions_counter)